# 针对专业人员的 TensorFlow 2.0 入门


###  将 Tensorflow 导入您的程序：

In [1]:
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
import numpy as np
import os
import gzip
from matplotlib import pyplot as plt 

### 加载并准备 MNIST 数据集。
在这一步中，会报错，因此将mnist数据集存储至本地。另寻方法加载。

In [4]:
# mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
def ym_load_data(data_folder):
    '''
    ---定义加载数据的函数，data_folder为保存gz数据的文件夹，该文件夹下有4个文件
    ---p:data_folder：数据集路径
    '''

    files = [
      'train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz',
      't10k-labels-idx1-ubyte.gz', 't10k-images-idx3-ubyte.gz'
    ]

    paths = []
    for fname in files:
        paths.append(os.path.join(data_folder,fname))

    with gzip.open(paths[0], 'rb') as lbpath:
        y_train = np.frombuffer(lbpath.read(), np.uint8, offset=8)

    with gzip.open(paths[1], 'rb') as imgpath:
        x_train = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_train), 28, 28)

    with gzip.open(paths[2], 'rb') as lbpath:
        y_test = np.frombuffer(lbpath.read(), np.uint8, offset=8)

    with gzip.open(paths[3], 'rb') as imgpath:
        x_test = np.frombuffer(
        imgpath.read(), np.uint8, offset=16).reshape(len(y_test), 28, 28)

    return (x_train, y_train), (x_test, y_test)

In [6]:
(x_train, y_train), (x_test, y_test) = ym_load_data(r'E:\a_code_place\data\mnist')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [43]:
# 增加一个通道维度
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

### 使用 tf.data 来将数据集切分为 batch 以及混淆数据集：

In [63]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(32)

### 使用 Keras 模型子类化（model subclassing） API 构建 tf.keras 模型：

In [114]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

### 为训练选择优化器与损失函数：

In [72]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### 选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果。

In [109]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

### 使用 tf.GradientTape 来训练模型：

In [116]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

### 测试模型：

In [111]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loos = loss_object(labels, predictions)
    
    test_loss(t_loos)
    test_accuracy(labels, predictions)

In [117]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # reset
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))
        
    

Epoch 1, Loss: 0.13752204179763794, Accuracy: 95.88500213623047, Test Loss: 0.06009141355752945, Test Accuracy: 97.89999389648438
Epoch 2, Loss: 0.04296257346868515, Accuracy: 98.66999816894531, Test Loss: 0.050236549228429794, Test Accuracy: 98.29999542236328
Epoch 3, Loss: 0.023720810189843178, Accuracy: 99.22500610351562, Test Loss: 0.0523027703166008, Test Accuracy: 98.47000122070312
Epoch 4, Loss: 0.013968967832624912, Accuracy: 99.53333282470703, Test Loss: 0.050709422677755356, Test Accuracy: 98.54999542236328
Epoch 5, Loss: 0.008814689703285694, Accuracy: 99.71666717529297, Test Loss: 0.06438631564378738, Test Accuracy: 98.3499984741211
